# Bibliotecas

In [365]:
# ! pip install selenium --quiet
# ! apt-get update --quiet
# ! apt install chromium-chromedriver --quiet

In [612]:
# from    selenium import webdriver
import  pandas as pd
# from  selenium import webdriver
import  numpy as np
import  re
import  plotly.express as px
import  plotly.graph_objects as go
import  matplotlib.pyplot as plt

from    plotly.subplots import make_subplots
from    sklearn.decomposition import PCA
from    sklearn.preprocessing import OneHotEncoder
from    sklearn.preprocessing import StandardScaler
from    scipy.stats import bartlett

In [367]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Webdriver

In [368]:
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')

# Nikola Jokic

In [369]:
# wd = webdriver.Chrome(options=options)

# temporadas_jokic = np.arange(2016, 2024)

# lista_dfs = []

# # Coleta os dados do jogador de 2016 a 2023
# for ano in temporadas_jokic:
#   site = 'https://www.espn.com.br/nba/jogador/jogo-a-jogo/_/id/3112335/tipo/nba/ano/' + str(ano)

#   wd.get(site)

#   html = wd.page_source

#   scrapping = pd.read_html(html)

#   df_stats_ano = pd.concat([scrapping[0],
#                             scrapping[1],
#                             scrapping[2],
#                             scrapping[3],
#                             scrapping[4],
#                             scrapping[5],
#                             scrapping[6]])

#   df_stats_ano.insert(0, 'Temporada', ano)

#   lista_dfs.append(df_stats_ano)

#   df_stats_jokic = pd.concat(lista_dfs).reset_index(drop=True)

In [370]:
# df_stats_jokic.to_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_raw.csv',
#                       encoding='utf-8',
#                       sep=',',
#                       index=False)

# Dataset raw from csv

In [371]:
df_stats_jokic = pd.read_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_raw.csv',
                             encoding='utf-8',
                             sep=',')

In [372]:
df_stats_jokic.reset_index(drop=True, inplace=True)

In [373]:
df_stats_jokic.head(6)

,Temporada,Date,ADV,Resultado,MIN,FG,FG%,3PT,3P%,FT,FT%,REB,V,BLK,STL,PF,TO,PTS,Estat. Temporada Regular
0,2016,Qui 14/4,emPOR,D107-99,35.0,7-10,70.0,1-2,50.0,3-3,100.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,NaN
1,2016,Dom 10/4,xUTAH,D100-84,36.0,7-9,77.8,0-2,0.0,5-6,83.3,11.0,4.0,1.0,2.0,4.0,1.0,19.0,NaN
2,2016,Sáb 9/4,xSA,V102-98,32.0,3-9,33.3,0-2,0.0,2-2,100.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,NaN
3,2016,Qua 6/4,xOKC,D124-102,19.0,3-5,60.0,1-3,33.3,0-0,0.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,NaN
4,2016,Dom 3/4,xSAC,D115-106,32.0,2-6,33.3,0-1,0.0,2-2,100.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,NaN
5,2016,april,april,april,30.8,4.4-7.8,56.4,0.4-2.0,20.0,2.4-2.6,92.3,10.8,4.0,1.2,0.8,2.6,0.6,11.6,NaN


# Data wrangling

In [374]:
df_stats_jokic.isnull().sum()

Temporada                     0
Date                          2
ADV                           2
Resultado                     2
MIN                           0
FG                            0
FG%                           0
3PT                           0
3P%                           0
FT                            0
FT%                           0
REB                           0
V                             0
BLK                           0
STL                           0
PF                            0
TO                            0
PTS                           0
Estat. Temporada Regular    654
dtype: int64

In [375]:
df_stats_jokic.drop(columns='Estat. Temporada Regular', inplace=True)

df_stats_jokic.dropna(axis=0, how='any', inplace=True)

In [376]:
df_stats_jokic.isnull().sum()

Temporada    0
Date         0
ADV          0
Resultado    0
MIN          0
FG           0
FG%          0
3PT          0
3P%          0
FT           0
FT%          0
REB          0
V            0
BLK          0
STL          0
PF           0
TO           0
PTS          0
dtype: int64

In [377]:
renamed_columns = {'Date': 'Data',
                   'V': 'Assistencias',
                   'PF': 'Faltas',
                   'TO': 'Tunovers',
                   'MIN': 'Minutos',
                   'REB': 'Rebotes',
                   'BLK': 'Bloqueios',
                   'STL': 'Desarmes',
                   'PTS': 'Pontos'}

df_stats_jokic.rename(columns=renamed_columns, inplace=True)

In [378]:
df_stats_jokic.head(6)

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos
0,2016,Qui 14/4,emPOR,D107-99,35.0,7-10,70.0,1-2,50.0,3-3,100.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0
1,2016,Dom 10/4,xUTAH,D100-84,36.0,7-9,77.8,0-2,0.0,5-6,83.3,11.0,4.0,1.0,2.0,4.0,1.0,19.0
2,2016,Sáb 9/4,xSA,V102-98,32.0,3-9,33.3,0-2,0.0,2-2,100.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0
3,2016,Qua 6/4,xOKC,D124-102,19.0,3-5,60.0,1-3,33.3,0-0,0.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0
4,2016,Dom 3/4,xSAC,D115-106,32.0,2-6,33.3,0-1,0.0,2-2,100.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0
5,2016,april,april,april,30.8,4.4-7.8,56.4,0.4-2.0,20.0,2.4-2.6,92.3,10.8,4.0,1.2,0.8,2.6,0.6,11.6


In [379]:
# Remoção de linhas com texto de nome de meses
df_stats_jokic = df_stats_jokic[(df_stats_jokic['Data']   != 'january')
                                & (df_stats_jokic['Data'] != 'february')
                                & (df_stats_jokic['Data'] != 'march')
                                & (df_stats_jokic['Data'] != 'april')
                                & (df_stats_jokic['Data'] != 'may')
                                & (df_stats_jokic['Data'] != 'june')
                                & (df_stats_jokic['Data'] != 'july')
                                & (df_stats_jokic['Data'] != 'august')
                                & (df_stats_jokic['Data'] != 'september')
                                & (df_stats_jokic['Data'] != 'october')
                                & (df_stats_jokic['Data'] != 'november')
                                & (df_stats_jokic['Data'] != 'december')]

df_stats_jokic.reset_index(drop=True, inplace=True)

In [380]:
# Verifica a remoção de linhas com texto de nome de meses
df_stats_jokic[(df_stats_jokic['Data']   == 'january')
               | (df_stats_jokic['Data'] == 'february')
               | (df_stats_jokic['Data'] == 'march')
               | (df_stats_jokic['Data'] == 'april')
               | (df_stats_jokic['Data'] == 'may')
               | (df_stats_jokic['Data'] == 'june')
               | (df_stats_jokic['Data'] == 'july')
               | (df_stats_jokic['Data'] == 'august')
               | (df_stats_jokic['Data'] == 'september')
               | (df_stats_jokic['Data'] == 'october')
               | (df_stats_jokic['Data'] == 'november')
               | (df_stats_jokic['Data'] == 'december')]

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos


In [381]:
# Regex de informações das colunas para novas colunas exclusivas
df_stats_jokic['FG_Tentativas'] = df_stats_jokic['FG'].str.extract(r'-(\d+)').astype(int)

df_stats_jokic['FG_Acertos'] = df_stats_jokic['FG'].str.extract(r'(\d+)-').astype(int)

df_stats_jokic['3PT_Tentativas'] = df_stats_jokic['3PT'].str.extract(r'-(\d+)').astype(int)

df_stats_jokic['3PT_Acertos'] = df_stats_jokic['3PT'].str.extract(r'(\d+)-').astype(int)

df_stats_jokic['FT_Tentativas'] = df_stats_jokic['FT'].str.extract(r'-(\d+)').astype(int)

df_stats_jokic['FT_Acertos'] = df_stats_jokic['FT'].str.extract(r'(\d+)-').astype(int)

df_stats_jokic['Adversario'] = df_stats_jokic['ADV'].str.extract(r'(?:em|x)([A-Z]+)')  # (?:em|x) - non-capturing group that matches 'em' or 'x'

df_stats_jokic['Placar'] = df_stats_jokic['Resultado'].str.extract(r'(\d+-\d+)')

df_stats_jokic['V_D'] = df_stats_jokic['Resultado'].str.extract(r'(^D|V)')

df_stats_jokic['Casa_Fora'] = df_stats_jokic['ADV'].str.extract(r'(em|x)')

df_stats_jokic['Dia_semana'] = df_stats_jokic['Data'].str.extract(r'([A-Za-z0-9_À-ÿ]+)')

df_stats_jokic['Mes'] = df_stats_jokic['Data'].str.extract(r'\b\d+/\b(\d+)')

<ipython-input-381-4812f5a36f9a>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-381-4812f5a36f9a>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-381-4812f5a36f9a>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-381-481

In [382]:
df_stats_jokic[(df_stats_jokic['Adversario'] == 'LEB')
               | (df_stats_jokic['Adversario'] == 'GIA')
               | (df_stats_jokic['Adversario'] == 'DUR')]  # Participações no All-Star

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Adversario,Placar,V_D,Casa_Fora,Dia_semana,Mes
252,2019,Seg 18/2,xLEB*,D178-164,13.0,3-4,75.0,0-0,0.0,0-0,0.0,9.0,0.0,0.0,0.0,1.0,1.0,6.0,4,3,0,0,0,0,LEB,178-164,D,x,Seg,2
327,2020,Seg 17/2,xGIA*,V157-155,12.0,2-4,50.0,1-3,33.3,0-0,0.0,2.0,1.0,0.0,1.0,1.0,0.0,5.0,4,2,3,1,0,0,GIA,157-155,V,x,Seg,2
419,2021,Seg 8/3,xDUR*,V170-150,19.0,3-4,75.0,0-0,0.0,0-0,0.0,7.0,5.0,1.0,0.0,1.0,2.0,6.0,4,3,0,0,0,0,DUR,170-150,V,x,Seg,3
478,2022,Seg 21/2,xDUR*,V163-160,21.0,4-5,80.0,2-3,66.7,0-0,0.0,9.0,8.0,0.0,1.0,0.0,1.0,10.0,5,4,3,2,0,0,DUR,163-160,V,x,Seg,2
549,2023,Seg 20/2,xGIA*,D184-175,20.0,2-4,50.0,0-1,0.0,0-0,0.0,5.0,6.0,0.0,2.0,0.0,2.0,4.0,4,2,1,0,0,0,GIA,184-175,D,x,Seg,2


In [383]:
df_stats_jokic = df_stats_jokic.replace('^LEB$', 'ALL', regex=True)
df_stats_jokic = df_stats_jokic.replace('^GIA$', 'ALL', regex=True)
df_stats_jokic = df_stats_jokic.replace('^DUR$', 'ALL', regex=True)

In [384]:
df_stats_jokic['Dia_semana'] = (df_stats_jokic['Dia_semana'].map({'Seg': 0,
                                                                  'Ter': 1,
                                                                  'Qua': 2,
                                                                  'Qui': 3,
                                                                  'Sex': 4,
                                                                  'Sáb': 5,
                                                                  'Dom': 6}))  # Dias da semana no padrão python

In [385]:
pd.set_option('display.max_columns', None)

df_stats_jokic['%FG'] = round(df_stats_jokic['FG_Acertos'] / df_stats_jokic['FG_Tentativas'], 2)

df_stats_jokic['%3PT'] = round(df_stats_jokic['3PT_Acertos'] / df_stats_jokic['3PT_Tentativas'], 2)

df_stats_jokic['%FT'] = round(df_stats_jokic['FT_Acertos'] / df_stats_jokic['FT_Tentativas'], 2)

df_stats_jokic['Casa_Fora'] = df_stats_jokic['Casa_Fora'].map({'em': 'fora','x': 'casa'})

## Análise do placar

In [386]:
# Cria colunas nos respectivos indexes
df_stats_jokic.insert(26, 'Placar_Denver', '')
df_stats_jokic.insert(27, 'Placar_Adversario', '')
df_stats_jokic.insert(28, 'Placar_Vencedor', '')
df_stats_jokic.insert(29, 'Placar_Perdedor', '')

In [387]:
df_stats_jokic['Placar_Vencedor'] = df_stats_jokic['Placar'].str.extract(r'(\d+)-').astype(int)

df_stats_jokic['Placar_Perdedor'] = df_stats_jokic['Placar'].str.extract(r'-(\d+)').astype(int)

In [388]:
"""
Identifica se o jogo foi vitória ou derrota.

O padrão do placar é sempre o nº de pontos do vencedor na frente.

Se o Denver ganhou, atribui o Placar_Vencedor para a coluna Placar_Denver
e Placar_Perdedor para Placar_Adversario.

Se o Denver perdeu é realizado o processo oposto.
"""
for i_resultado, resultado in enumerate(df_stats_jokic['V_D']):

    if resultado == 'D':
        df_stats_jokic['Placar_Denver'].loc[i_resultado] = df_stats_jokic['Placar_Perdedor'].loc[i_resultado]
        df_stats_jokic['Placar_Adversario'].loc[i_resultado] = df_stats_jokic['Placar_Vencedor'].loc[i_resultado]

    else:
        df_stats_jokic['Placar_Denver'].loc[i_resultado] = df_stats_jokic['Placar_Vencedor'].loc[i_resultado]
        df_stats_jokic['Placar_Adversario'].loc[i_resultado] = df_stats_jokic['Placar_Perdedor'].loc[i_resultado]

<ipython-input-388-4f5b2557e00e>:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-388-4f5b2557e00e>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-388-4f5b2557e00e>:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-388-4f5b2557e00e>:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.

In [389]:
df_stats_jokic.head()

,Temporada,Data,ADV,Resultado,Minutos,FG,FG%,3PT,3P%,FT,FT%,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Adversario,Placar,Placar_Denver,Placar_Adversario,Placar_Vencedor,Placar_Perdedor,V_D,Casa_Fora,Dia_semana,Mes,%FG,%3PT,%FT
0,2016,Qui 14/4,emPOR,D107-99,35.0,7-10,70.0,1-2,50.0,3-3,100.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,10,7,2,1,3,3,POR,107-99,99,107,107,99,D,fora,3,4,0.70,0.50,1.00
1,2016,Dom 10/4,xUTAH,D100-84,36.0,7-9,77.8,0-2,0.0,5-6,83.3,11.0,4.0,1.0,2.0,4.0,1.0,19.0,9,7,2,0,6,5,UTAH,100-84,84,100,100,84,D,casa,6,4,0.78,0.00,0.83
2,2016,Sáb 9/4,xSA,V102-98,32.0,3-9,33.3,0-2,0.0,2-2,100.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,9,3,2,0,2,2,SA,102-98,102,98,102,98,V,casa,5,4,0.33,0.00,1.00
3,2016,Qua 6/4,xOKC,D124-102,19.0,3-5,60.0,1-3,33.3,0-0,0.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,5,3,3,1,0,0,OKC,124-102,102,124,124,102,D,casa,2,4,0.60,0.33,NaN
4,2016,Dom 3/4,xSAC,D115-106,32.0,2-6,33.3,0-1,0.0,2-2,100.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,6,2,1,0,2,2,SAC,115-106,106,115,115,106,D,casa,6,4,0.33,0.00,1.00


## Identificando as conferências dos times

In [390]:
conferencia_leste = ['ATL', 'BOS', 'BKN', 'CHA', 'CHI', 'CLE', 'DET', 'IND', 'MIA', 'MIL', 'NY', 'ORL', 'PHI', 'TOR', 'WSH']
conferencia_oeste = ['DAL', 'DEN', 'GS', 'HOU', 'LAC', 'LAL', 'MEM', 'MIN', 'NO', 'OKC', 'PHX', 'POR', 'SAC', 'SA', 'UTAH']

In [391]:
df_stats_jokic.insert(25, 'Conferencia', '')  # Cria a coluna Liga no index 16

In [392]:
def conferencia(time):
    """
    Identifica a qual liga o time pertence
    """
    if time in conferencia_leste:
        return 'Leste'
    elif time in conferencia_oeste:
        return 'Oeste'
    else:
        return '-'

In [393]:
# series.apply aplica a função sobre a série Adversario
# Depois a respectiva liga é inserida na coluna Conferencia
df_stats_jokic['Conferencia'] = df_stats_jokic['Adversario'].apply(conferencia)

In [394]:
df_stats_jokic.drop(columns=['Data', 'ADV', 'Resultado', 'FG', 'FG%', '3PT',
                             '3P%', 'FT', 'FT%', 'Placar', 'Placar_Vencedor',
                             'Placar_Perdedor'], inplace=True)

df_stats_jokic.head()

,Temporada,Minutos,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Adversario,Conferencia,Placar_Denver,Placar_Adversario,V_D,Casa_Fora,Dia_semana,Mes,%FG,%3PT,%FT
0,2016,35.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,10,7,2,1,3,3,POR,Oeste,99,107,D,fora,3,4,0.70,0.50,1.00
1,2016,36.0,11.0,4.0,1.0,2.0,4.0,1.0,19.0,9,7,2,0,6,5,UTAH,Oeste,84,100,D,casa,6,4,0.78,0.00,0.83
2,2016,32.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,9,3,2,0,2,2,SA,Oeste,102,98,V,casa,5,4,0.33,0.00,1.00
3,2016,19.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,5,3,3,1,0,0,OKC,Oeste,102,124,D,casa,2,4,0.60,0.33,NaN
4,2016,32.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,6,2,1,0,2,2,SAC,Oeste,106,115,D,casa,6,4,0.33,0.00,1.00


## Salva dataset ajustado

In [395]:
# df_stats_jokic.to_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_ajustado.csv',
#                       encoding='utf-8',
#                       sep=',',
#                       index=False)

# Dataset from csv

In [396]:
df_stats_jokic = pd.read_csv('/content/drive/MyDrive/NBA Analysis/nba_stats_jokic_ajustado.csv',
                             encoding='utf-8',
                             sep=',')

df_stats_jokic = df_stats_jokic[df_stats_jokic['Adversario'] != 'ALL'].copy()  # Remova as linhas com dados do All-Star

In [397]:
df_stats_jokic['V_D'].value_counts()

V    351
D    245
Name: V_D, dtype: int64

In [614]:
df_stats_jokic.fillna(0, inplace=True)
df_stats_jokic.isnull().sum()

Temporada            0
Minutos              0
Rebotes              0
Assistencias         0
Bloqueios            0
Desarmes             0
Faltas               0
Tunovers             0
Pontos               0
FG_Tentativas        0
FG_Acertos           0
3PT_Tentativas       0
3PT_Acertos          0
FT_Tentativas        0
FT_Acertos           0
Adversario           0
Conferencia          0
Placar_Denver        0
Placar_Adversario    0
V_D                  0
Casa_Fora            0
Dia_semana           0
Mes                  0
%FG                  0
%3PT                 0
%FT                  0
dtype: int64

# Análise gráfica

## %stats

In [399]:
media_porcent_temporadas = df_stats_jokic[['Temporada', '%FG', '%FT', '%3PT']].groupby('Temporada').mean()

In [400]:
fig = px.line(media_porcent_temporadas,
              x=media_porcent_temporadas.index,
              y=['%FG', '%FT', '%3PT'],
              labels={'variable': ''})

fig.update_layout(height=400,
                  width=1000,
                  showlegend=True,
                  title='Jokic %stats',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

fig.update_xaxes(title_text=None)
fig.update_yaxes(title_text=None)
fig.show()

## Pontos

In [401]:
media_pontos_temporadas = df_stats_jokic[['Temporada', 'Pontos']].groupby('Temporada').mean()

media_pontos_temporadas_por_tipo = df_stats_jokic[['Temporada', 'FG_Acertos', 'FT_Acertos', '3PT_Acertos']].groupby('Temporada').mean()

In [402]:
media_pontos_temporadas_por_tipo['FG_Pontos'] = media_pontos_temporadas_por_tipo['FG_Acertos'] * 2

media_pontos_temporadas_por_tipo['FT_Pontos'] = media_pontos_temporadas_por_tipo['FT_Acertos'] * 1

media_pontos_temporadas_por_tipo['3_Pontos'] = media_pontos_temporadas_por_tipo['3PT_Acertos'] * 3

media_pontos_temporadas_por_tipo

,FG_Acertos,FT_Acertos,3PT_Acertos,FG_Pontos,FT_Pontos,3_Pontos
Temporada,,,,,,
2016,3.837500,1.925000,0.350000,7.675000,1.925000,1.050000
2017,6.767123,2.575342,0.616438,13.534247,2.575342,1.849315
2018,6.720000,3.546667,1.480000,13.440000,3.546667,4.440000
2019,7.700000,3.612500,1.037500,15.400000,3.612500,3.112500
2020,7.739726,3.369863,1.095890,15.479452,3.369863,3.287671
2021,10.166667,4.750000,1.277778,20.333333,4.750000,3.833333
2022,10.324324,5.121622,1.310811,20.648649,5.121622,3.932432
2023,9.362319,4.942029,0.826087,18.724638,4.942029,2.478261


In [403]:
fig = make_subplots(shared_xaxes=True)

fig.add_trace(go.Scatter(x=media_pontos_temporadas_por_tipo.index,
                         y=media_pontos_temporadas_por_tipo['FG_Pontos'],
                         mode='lines',
                         line=dict(color='#845EC2'),
                         name='FG'))

fig.add_trace(go.Scatter(x=media_pontos_temporadas_por_tipo.index,
                         y=media_pontos_temporadas_por_tipo['FT_Pontos'],
                         mode='lines',
                         line=dict(color='#008BC9'),
                         name='FT'))

fig.add_trace(go.Scatter(x=media_pontos_temporadas_por_tipo.index,
                         y=media_pontos_temporadas_por_tipo['3_Pontos'],
                         mode='lines',
                         line=dict(color='#FF8066'),
                         name='3PT'))

fig.add_trace(go.Scatter(x=media_pontos_temporadas.index,
                         y=media_pontos_temporadas['Pontos'],
                         mode='lines',
                         line=dict(color='#936C00'),
                         name='Média geral'))

fig.update_layout(height=400,
                  width=1000,
                  showlegend=True,
                  title='Jokic - média de pontos nas temporadas',
                  margin=dict(l=50,
                              r=50,
                              b=50,
                              t=50))

fig.update_xaxes(title_text=None)

fig.update_yaxes(title_text=None)

fig.show()

## Correlação

In [404]:
correlacao = df_stats_jokic[['Temporada',
                             'Minutos',
                             'Rebotes',
                             'Assistencias',
                             'Bloqueios',
                             'Desarmes',
                             'Faltas',
                             'Tunovers',
                             'Pontos',
                             'FG_Tentativas',
                             'FG_Acertos',
                             '3PT_Tentativas',
                             '3PT_Acertos',
                             'FT_Tentativas',
                             'FT_Acertos',
                             'Dia_semana',
                             'Mes',
                             '%FG',
                             '%3PT',
                             '%FT']].corr().round(2)

fig = px.imshow(correlacao, text_auto=True, aspect='auto', color_continuous_scale='RdBu_r')

fig.update_layout(margin=dict(t=80,
                              r=50,
                              b=50,
                              l=50),
                  showlegend=False,
                  width=1200,
                  height=1200)

fig.update_xaxes(side='top')

fig.show()

±0.9 → correlação muito forte

±0.7 a ±0.9 → correlação forte

±0.5 a ±0.7 → correlação moderada

±0.3 a ±0.5 → correlação fraca

### Correlação com pontos

#### Correlação muito forte

In [405]:
correlacao.loc[correlacao['Pontos'] >= 0.9].index

Index(['Pontos', 'FG_Acertos'], dtype='object')

#### Correlação forte

In [406]:
correlacao.loc[(correlacao['Pontos'] >= 0.7)
               & (correlacao['Pontos'] < 0.9)].index

Index(['FG_Tentativas'], dtype='object')

#### Correlação moderada

In [407]:
correlacao.loc[(correlacao['Pontos'] >= 0.5)
               & (correlacao['Pontos'] < 0.7)].index

Index(['Temporada', 'Minutos', 'FT_Tentativas', 'FT_Acertos'], dtype='object')

#### Correlação fraca

In [408]:
correlacao.loc[(correlacao['Pontos'] >= 0.3)
               & (correlacao['Pontos'] < 0.5)].index

Index(['Rebotes', 'Assistencias', '3PT_Tentativas', '3PT_Acertos', '%FG'], dtype='object')

### Conclusões

#### Pontuação

Os pontos do Jokic tem correlação:
* Muito forte com lances de quadra (FG)
* Moderada com lances livres (FT)
* Fraca com lances de 3 pontos (3PT)

In [409]:
media_acertos_fg = df_stats_jokic['FG_Acertos'].mean().round(2)
media_acertos_ft = df_stats_jokic['FT_Acertos'].mean().round(2)
media_acertos_3pt = df_stats_jokic['3PT_Acertos'].mean().round(2)

print('Média de acertos FG por jogo:', media_acertos_fg)
print('Média de acertos FT por jogo:', media_acertos_ft)
print('Média de acertos 3PT por jogo:', media_acertos_3pt)

Média de acertos FG por jogo: 7.77
Média de acertos FT por jogo: 3.7
Média de acertos 3PT por jogo: 0.99


#### Aproveitamento

In [410]:
media_aproveitamento_porcent_fg = (df_stats_jokic['%FG'].mean() * 100).round(2)
media_aproveitamento_porcent_ft = (df_stats_jokic['%FT'].mean() * 100).round(2)
media_aproveitamento_porcent_3pt = (df_stats_jokic['%3PT'].mean() * 100).round(2)

print('Média de aproveitamento FG por jogo:', media_aproveitamento_porcent_fg, '%')
print('Média de aproveitamento FT por jogo:', media_aproveitamento_porcent_ft, '%')
print('Média de aproveitamento 3PT por jogo:', media_aproveitamento_porcent_3pt, '%')

Média de aproveitamento FG por jogo: 55.69 %
Média de aproveitamento FT por jogo: 82.0 %
Média de aproveitamento 3PT por jogo: 35.07 %


A maior porcentagem de aproveitamento é de lances livres (FT)

Apesar do bom aproveitamento, a correlação é mais forte com FG por valer mais pontos que FT

#### Média de pontos

In [411]:
pontos_fg = 2
pontos_ft = 1
pontos_3pt = 3

media_pontos_fg = (df_stats_jokic['FG_Acertos'].mean() * pontos_fg).round(2)
media_pontos_ft = (df_stats_jokic['FT_Acertos'].mean() * pontos_ft).round(2)
media_pontos_3pt = (df_stats_jokic['3PT_Acertos'].mean() * pontos_3pt).round(2)
media_pontos_por_jogo = (df_stats_jokic['Pontos'].mean()).round(2)

print('Média de pontos FG:', media_pontos_fg)
print('Média de pontos FT:', media_pontos_ft)
print('Média de pontos 3PT:', media_pontos_3pt)
print('Média de pontos por jogo:', media_pontos_por_jogo)

Média de pontos FG: 15.53
Média de pontos FT: 3.7
Média de pontos 3PT: 2.98
Média de pontos por jogo: 20.22


# Substituição de variáveis categóricas

## Dummies

In [615]:
df_stats_jokic_for_dummies = df_stats_jokic.copy()
df_stats_jokic_for_dummies.reset_index(drop=True, inplace=True)

In [617]:
df_stats_jokic_dummies = pd.get_dummies(df_stats_jokic_for_dummies, prefix='', prefix_sep='')
df_stats_jokic_dummies

,Temporada,Minutos,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Placar_Denver,Placar_Adversario,Dia_semana,Mes,%FG,%3PT,%FT,ATL,BKN,BOS,CHA,CHI,CLE,DAL,DET,GS,HOU,IND,LAC,LAL,MEM,MIA,MIL,MIN,NO,NY,OKC,ORL,PHI,PHX,POR,SA,SAC,TOR,UTAH,WSH,Leste,Oeste,D,V,casa,fora
0,2016,35.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,10,7,2,1,3,3,99,107,3,4,0.70,0.50,1.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1
1,2016,36.0,11.0,4.0,1.0,2.0,4.0,1.0,19.0,9,7,2,0,6,5,84,100,6,4,0.78,0.00,0.83,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0
2,2016,32.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,9,3,2,0,2,2,102,98,5,4,0.33,0.00,1.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1,0
3,2016,19.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,5,3,3,1,0,0,102,124,2,4,0.60,0.33,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
4,2016,32.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,6,2,1,0,2,2,106,115,6,4,0.33,0.00,1.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,2023,35.0,13.0,9.0,0.0,4.0,3.0,3.0,31.0,17,12,4,0,7,7,110,99,3,10,0.71,0.00,1.00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0
592,2023,27.0,9.0,9.0,0.0,0.0,5.0,1.0,9.0,4,3,0,0,3,3,110,135,1,10,0.75,0.00,1.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1
593,2023,39.0,16.0,13.0,1.0,1.0,2.0,3.0,19.0,10,6,1,1,9,6,122,117,6,10,0.60,1.00,0.67,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0
594,2023,34.0,12.0,10.0,0.0,0.0,4.0,5.0,26.0,13,7,2,1,11,11,128,123,5,10,0.54,0.50,1.00,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1


### Correlação dummies

In [ ]:
correlacao_dummies = df_stats_jokic_dummies.corr().round(2)

fig = px.imshow(correlacao_dummies, text_auto=True, aspect='auto', color_continuous_scale='RdBu_r')

fig.update_layout(margin=dict(t=80,
                              r=50,
                              b=50,
                              l=50),
                  showlegend=False,
                  width=1800,
                  height=1800)

fig.update_xaxes(side='top')

fig.show()

# Principal Component Analysis (PCA)

## Teste de Esfericidade de Bartlett

Hipótese Nula que a matriz de correlação ρ da sua base de dados é estatisticamente igual a uma matriz identidade.

*   Significa que temos uma base de dados onde não existe correlação entre nenhuma das variáveis que compõem essa base. Importante salientar que o teste verifica se as correlações entre as variáveis são ESTATISTICAMENTE IGUAIS a 0, o que não quer dizer que elas sejam 0.


Hipótese Alternativa, temos que a matriz de correlações da sua base de dados não é ESTATISTICAMENTE IGUAL a uma matriz identidade.

Para **rejeitarmos** a Hipótese Nula, que é o que queremos, temos de ter que o valor calculado na distribuição qui quadrada acima deve ser maior que o valor dessa mesma distribuição para o nível de significância desejado. Se for maior, podemos rejeitar a hipótese nula e aceitarmos a hipótese alternativa.

Fonte: [Abrindo a Caixa Preta: PCA (Análise de Componentes Principais)](https://medium.com/@felipeverasaraujo/abrindo-a-caixa-preta-pca-an%C3%A1lise-de-componentes-principais-d5d400781dfe)

In [441]:
stats, p_value = bartlett(df_stats_jokic_dummies['Temporada'], df_stats_jokic_dummies['Minutos'], df_stats_jokic_dummies['Rebotes'], df_stats_jokic_dummies['Assistencias'], df_stats_jokic_dummies['Bloqueios'],
                          df_stats_jokic_dummies['Desarmes'], df_stats_jokic_dummies['Faltas'], df_stats_jokic_dummies['Tunovers'], df_stats_jokic_dummies['Pontos'], df_stats_jokic_dummies['FG_Tentativas'],
                          df_stats_jokic_dummies['FG_Acertos'], df_stats_jokic_dummies['3PT_Tentativas'], df_stats_jokic_dummies['3PT_Acertos'], df_stats_jokic_dummies['FT_Tentativas'],
                          df_stats_jokic_dummies['FT_Acertos'], df_stats_jokic_dummies['Placar_Denver'], df_stats_jokic_dummies['Placar_Adversario'], df_stats_jokic_dummies['Dia_semana'], df_stats_jokic_dummies['Mes'],
                          df_stats_jokic_dummies['%FG'], df_stats_jokic_dummies['%3PT'], df_stats_jokic_dummies['%FT'], df_stats_jokic_dummies['ATL'], df_stats_jokic_dummies['BKN'], df_stats_jokic_dummies['BOS'], df_stats_jokic_dummies['CHA'],
                          df_stats_jokic_dummies['CHI'], df_stats_jokic_dummies['CLE'], df_stats_jokic_dummies['DAL'], df_stats_jokic_dummies['DET'], df_stats_jokic_dummies['GS'], df_stats_jokic_dummies['HOU'], df_stats_jokic_dummies['IND'],
                          df_stats_jokic_dummies['LAC'], df_stats_jokic_dummies['LAL'], df_stats_jokic_dummies['MEM'], df_stats_jokic_dummies['MIA'], df_stats_jokic_dummies['MIL'], df_stats_jokic_dummies['MIN'],df_stats_jokic_dummies['NO'],
                          df_stats_jokic_dummies['NY'], df_stats_jokic_dummies['OKC'], df_stats_jokic_dummies['ORL'], df_stats_jokic_dummies['PHI'], df_stats_jokic_dummies['PHX'], df_stats_jokic_dummies['POR'], df_stats_jokic_dummies['SA'],
                          df_stats_jokic_dummies['SAC'], df_stats_jokic_dummies['TOR'],df_stats_jokic_dummies['UTAH'], df_stats_jokic_dummies['WSH'], df_stats_jokic_dummies['casa'], df_stats_jokic_dummies['fora'], df_stats_jokic_dummies['D'],
                          df_stats_jokic_dummies['V'], df_stats_jokic_dummies['Leste'], df_stats_jokic_dummies['Oeste'])

In [442]:
significancia = 0.05

In [443]:
p_value

0.0

In [444]:
if p_value < significancia:
    print('Hipótese nula rejeitada\nÉ possível utilizar PCA')
else:
    print('Hipótese nula aceita\nNão é possível utilizar PCA')

Hipótese nula rejeitada
É possível utilizar PCA


## PCA

1. Padronização dos dados
2. Obtenção dos autovalores e autovetores
3. Obtenção das componentes principais
4. Correlação entre valores padronizados e scores dos componentes principais

In [618]:
df_stats_jokic_pca = df_stats_jokic.copy()

In [621]:
df_stats_jokic_pca.drop(columns=['Adversario', 'Conferencia', 'V_D', 'Casa_Fora'], inplace=True)

In [622]:
df_stats_jokic_pca

,Temporada,Minutos,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Placar_Denver,Placar_Adversario,Dia_semana,Mes,%FG,%3PT,%FT
0,2016,35.0,13.0,5.0,0.0,0.0,3.0,2.0,18.0,10,7,2,1,3,3,99,107,3,4,0.70,0.50,1.00
1,2016,36.0,11.0,4.0,1.0,2.0,4.0,1.0,19.0,9,7,2,0,6,5,84,100,6,4,0.78,0.00,0.83
2,2016,32.0,15.0,4.0,1.0,1.0,2.0,0.0,8.0,9,3,2,0,2,2,102,98,5,4,0.33,0.00,1.00
3,2016,19.0,2.0,2.0,2.0,0.0,2.0,0.0,7.0,5,3,3,1,0,0,102,124,2,4,0.60,0.33,0.00
4,2016,32.0,13.0,5.0,2.0,1.0,2.0,0.0,6.0,6,2,1,0,2,2,106,115,6,4,0.33,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,2023,35.0,13.0,9.0,0.0,4.0,3.0,3.0,31.0,17,12,4,0,7,7,110,99,3,10,0.71,0.00,1.00
597,2023,27.0,9.0,9.0,0.0,0.0,5.0,1.0,9.0,4,3,0,0,3,3,110,135,1,10,0.75,0.00,1.00
598,2023,39.0,16.0,13.0,1.0,1.0,2.0,3.0,19.0,10,6,1,1,9,6,122,117,6,10,0.60,1.00,0.67
599,2023,34.0,12.0,10.0,0.0,0.0,4.0,5.0,26.0,13,7,2,1,11,11,128,123,5,10,0.54,0.50,1.00


### 1. Padronização dos dados

In [625]:
scaler = StandardScaler()
stats_jokic_pca_scaled = scaler.fit_transform(df_stats_jokic_pca)

In [626]:
stats_jokic_pca_scaled

array([[-1.49992572,  0.61237935,  0.58865305, ...,  0.88830621,
         0.63093966,  0.83544045],
       [-1.49992572,  0.75692512,  0.11294157, ...,  1.37224635,
        -0.93811665,  0.34581281],
       [-1.49992572,  0.17874201,  1.06436454, ..., -1.34991689,
        -0.93811665,  0.83544045],
       ...,
       [ 1.56456503,  1.19056246,  1.30222028, ...,  0.28338105,
         2.19999596, -0.1150132 ],
       [ 1.56456503,  0.46783357,  0.35079731, ..., -0.07957405,
         0.63093966,  0.83544045],
       [ 1.56456503,  0.32328779, -1.55204863, ...,  0.94879873,
         0.09746051,  0.83544045]])

In [628]:
columns = df_stats_jokic_pca.columns
data = stats_jokic_pca_scaled

df_stats_jokic_pca_scaled = pd.DataFrame(data=data, columns=columns)

df_stats_jokic_pca_scaled

,Temporada,Minutos,Rebotes,Assistencias,Bloqueios,Desarmes,Faltas,Tunovers,Pontos,FG_Tentativas,FG_Acertos,3PT_Tentativas,3PT_Acertos,FT_Tentativas,FT_Acertos,Placar_Denver,Placar_Adversario,Dia_semana,Mes,%FG,%3PT,%FT
0,-1.499926,0.612379,0.588653,-0.456321,-0.808934,-1.071772,0.177349,-0.449204,-0.233011,-0.660325,-0.202203,-0.387953,0.004596,-0.424081,-0.237025,-0.952446,-0.175231,-0.143102,-0.383757,0.888306,0.630940,0.835440
1,-1.499926,0.756925,0.112942,-0.734121,0.330841,0.717517,0.896395,-0.967049,-0.128279,-0.823829,-0.202203,-0.387953,-0.908413,0.444483,0.440514,-2.119825,-0.772506,1.398261,-0.383757,1.372246,-0.938117,0.345813
2,-1.499926,0.178742,1.064365,-0.734121,0.330841,-0.177128,-0.541698,-1.484893,-1.280332,-0.823829,-1.259335,-0.387953,-0.908413,-0.713602,-0.575794,-0.718970,-0.943155,0.884474,-0.383757,-1.349917,-0.938117,0.835440
3,-1.499926,-1.700353,-2.027760,-1.289723,1.470616,-1.071772,-0.541698,-1.484893,-1.385064,-1.477845,-1.259335,0.065419,0.004596,-1.292644,-1.253332,-0.718970,1.275293,-0.656890,-0.383757,0.283381,0.097461,-2.044722
4,-1.499926,0.178742,0.588653,-0.456321,1.470616,-0.177128,-0.541698,-1.484893,-1.489797,-1.314341,-1.523619,-0.841325,-0.908413,-0.713602,-0.575794,-0.407669,0.507368,1.398261,-0.383757,-1.349917,-0.938117,0.835440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,1.564565,0.612379,0.588653,0.654883,-0.808934,2.506807,0.177349,0.068640,1.128506,0.484202,1.119212,0.518792,-0.908413,0.734004,1.118052,-0.096368,-0.857830,-0.143102,0.970946,0.948799,-0.938117,0.835440
592,1.564565,-0.543987,-0.362770,0.654883,-0.808934,-1.071772,1.615441,-0.967049,-1.175600,-1.641348,-1.259335,-1.294697,-0.908413,-0.424081,-0.237025,-0.096368,2.213867,-1.170677,0.970946,1.190769,-0.938117,0.835440
593,1.564565,1.190562,1.302220,1.766086,0.330841,-0.177128,-0.541698,0.068640,-0.128279,-0.660325,-0.466486,-0.841325,0.004596,1.313047,0.779283,0.837536,0.678018,1.398261,0.970946,0.283381,2.199996,-0.115013
594,1.564565,0.467834,0.350797,0.932684,-0.808934,-1.071772,0.896395,1.104330,0.604845,-0.169814,-0.202203,-0.387953,0.004596,1.892089,2.473129,1.304488,1.189968,0.884474,0.970946,-0.079574,0.630940,0.835440


In [638]:
pca = PCA(random_state=0)
pca = pca.fit(df_stats_jokic_pca_scaled)

In [639]:
fig = px.line(x=range(1, len(pca.explained_variance_) + 1),
              y=pca.explained_variance_,
              labels={'x': 'N de componentes', 'y': 'Eigenvalues'})

fig.update_layout(margin=dict(t=50,
                              r=50,
                              b=50,
                              l=50),
                  showlegend=False,
                  width=1000,
                  height=400)

fig.add_hline(y=1, line_width=2, line_dash='dash', line_color='red')

fig.show()

* **Autovalores**: Para o universo da PCA, podemos dizer que os autovalores mostram quantas variáveis cada autovalor representa em termos de variância. Quando dividimos os Autovalores pelo número de variáveis na base de dados, conseguimos entender a variância explicada que cada fator terá sobre a variância total das variáveis originais.

* **Autovetores**: Para o universo da PCA, podemos definir os autovetores como sendo as projeções das variâncias capturadas das variáveis originais. Ele vai mostrar o quanto cada fator está capturando da variância de cada variável.

*Obs: Enquanto o autovalor vai mostrar o quanto cada fator explica da variância TOTAL das variáveis, os autovetores vão mostrar o quanto da variância DE CADA variável está sendo explicada por cada fator.*

Fonte: [Abrindo a Caixa Preta: PCA (Análise de Componentes Principais)](https://medium.com/@felipeverasaraujo/abrindo-a-caixa-preta-pca-an%C3%A1lise-de-componentes-principais-d5d400781dfe)

### 2. Obtenção dos autovalores e autovetores

In [658]:
matriz_autovalores = np.round(pca.explained_variance_, 2)  # Autovalores
# Variância explicada por cada uma das componentes principais
matriz_autovalores

array([6.24, 2.1 , 1.59, 1.46, 1.19, 1.09, 1.04, 0.99, 0.95, 0.93, 0.85,
       0.78])

In [647]:
(pca.explained_variance_).sum()  # Variância total do conjunto de dados padronizados (após normalização)

22.03697478991595

In [657]:
matriz_autovetores = pca.components_

In [649]:
np.round(pca.explained_variance_ratio_, 2)
# Cada valor é o resultado da divisão de cada autovalor pela soma dos autovalores

array([0.28, 0.1 , 0.07, 0.07, 0.05, 0.05, 0.05, 0.04, 0.04, 0.04, 0.04,
       0.04, 0.03, 0.03, 0.02, 0.02, 0.02, 0.01, 0.  , 0.  , 0.  , 0.  ])

In [644]:
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)

fig = px.area(x=range(1, exp_var_cumul.shape[0] + 1),
              y=exp_var_cumul,
              labels={'x': 'N de Components', 'y': 'Explained Variance'})

fig.update_layout(yaxis=dict(tickmode='linear',
                               tick0=0,
                               dtick=0.1),
                  xaxis=dict(tickmode='linear',
                               tick0=0,
                               dtick=5),
                  margin=dict(t=50,
                              r=50,
                              b=50,
                              l=50),
                  showlegend=False,
                  width=1000,
                  height=400)

fig.add_hline(y=0.8, line_width=2, line_dash='dash', line_color='green')

fig.add_hline(y=0.9, line_width=2, line_dash='dash', line_color='red')

fig.show()

In [650]:
exp_var_cumul_series = pd.Series(exp_var_cumul)

exp_var_cumul_series_90_porcent = exp_var_cumul_series[exp_var_cumul_series >= 0.9]

n_components = exp_var_cumul_series_90_porcent.index[0]
# Número mínimo de componentes principais que explicam 90%
# da variação do conjunto de dados
n_components

12


### Nº de componentes definido

In [685]:
nomes_colunas = {}

for i, componente_principal in enumerate(range(1, n_components + 1)):
    nome_componente = 'CP' + str(componente_principal)
    colunas = {i: nome_componente}
    nomes_colunas.update(colunas)

nomes_colunas

{0: 'CP1',
 1: 'CP2',
 2: 'CP3',
 3: 'CP4',
 4: 'CP5',
 5: 'CP6',
 6: 'CP7',
 7: 'CP8',
 8: 'CP9',
 9: 'CP10',
 10: 'CP11',
 11: 'CP12'}

In [686]:
pca_n_components = PCA(random_state=0, n_components=n_components)
pca_n_components = pca.fit(df_stats_jokic_pca_scaled)

In [687]:
matriz_autovetores_n_components = pca.components_

### 3. Obtenção das componentes principais

In [690]:
matrix_componentes_principais = np.matmul(df_stats_jokic_pca_scaled, matriz_autovetores_n_components.T)
matrix_componentes_principais.rename(columns=nomes_colunas)

,CP1,CP2,CP3,CP4,CP5,CP6,CP7,CP8,CP9,CP10,CP11,CP12
0,0.810909,0.222598,0.599002,-0.416180,0.167484,0.024165,0.551118,0.694852,-0.459176,-0.732603,-1.510855,-0.067789
1,1.010953,-0.554240,-1.099294,-1.296149,-0.175872,-0.684422,2.219582,1.265652,1.065257,-0.196518,-0.697022,0.286508
2,2.595518,0.000858,-1.755127,-0.583514,-0.519645,-0.498247,0.616972,1.530108,-0.887954,-0.237717,0.621560,-0.532605
3,4.242031,1.013975,1.761732,-0.507617,-0.247066,-2.117894,-0.922110,-0.495712,-0.677155,1.299165,-0.014651,0.097907
4,2.731828,-0.632068,-1.094373,-0.632713,-1.544918,-1.097216,0.536321,1.088245,-1.187414,0.588980,1.386418,-0.048696
...,...,...,...,...,...,...,...,...,...,...,...,...
591,-2.395012,-1.218876,-1.356959,0.148148,1.625177,1.007690,0.563052,0.457369,2.277217,-0.116442,0.856931,-0.057728
592,1.612305,-2.606065,1.236826,0.604445,-0.424563,0.779087,-0.919607,-1.800652,0.504324,0.102072,-0.275583,-1.001001
593,-1.794310,-1.258470,0.925180,1.152304,-0.416300,1.636454,1.617942,-0.339574,-1.318859,0.646295,1.163784,-0.554767
594,-2.520754,-2.391739,0.629248,-0.609995,-0.959693,1.814298,0.671647,-1.948140,-0.895585,-0.802094,0.212565,-0.401827


### 4. Correlação entre valores padronizados e scores dos componentes principais

In [709]:
# Calculate correlations between each column of df_stats_jokic_pca_scaled and matrix_componentes_principais
correlations = pd.DataFrame(index=df_stats_jokic_pca_scaled.columns, columns=matrix_componentes_principais.columns)

for col1 in df_stats_jokic_pca_scaled.columns:
    for col2 in matrix_componentes_principais.columns:
        correlation_value = np.corrcoef(df_stats_jokic_pca_scaled[col1], matrix_componentes_principais[col2])[0, 1]
        correlations.loc[col1, col2] = correlation_value.round(2)

# Convert the correlation values to numeric type
correlations = correlations.apply(pd.to_numeric)

In [710]:
df_correlacoes = pd.DataFrame(correlations)
df_correlacoes.rename(columns=nomes_colunas, inplace=True)

fig = px.imshow(df_correlacoes, text_auto=True, aspect='auto', color_continuous_scale='RdBu_r')

fig.update_layout(margin=dict(t=80,
                              r=50,
                              b=50,
                              l=50),
                  showlegend=False,
                  width=1200,
                  height=1200)

fig.update_xaxes(side='top')

fig.show()